# The evolution of compositionality under environmental noise



### Bigger picture:

**We hypothesise that** other-initiated repair and a compositional language co-evolve under the following assumptions:

1. There is a pressure for communicative success
    2. Understanding part of the meaning is better than understanding none of it
3. There is a pressure to make repair sequences as efficient as possible (in terms of utterance length)
    4. Listeners have at least two types of repair initiator at their disposal: (i) open request, and (ii) closed request
    
This hypothesis is based on the idea that 



### Step 1: Adding noise to existing model of evolution of compositional language

Here we adapt the model of Kirby et al. (2015)---where they show that a compositional language evolves when there is a pressure for both (i) learnability and (ii) expressivity---to incorporate noise.




#### References  
Kirby, S., Tamariz, M., Cornish, H., & Smith, K. (2015). Compression and communication in the cultural evolution of linguistic structure. Cognition, 141, 87–102. https://doi.org/10.1016/j.cognition.2015.03.016

## Production in the Kirby et al. (2015) model:

$$
P(f \mid l, t) \propto \Bigg\{
\begin{array}{ll}
(\frac{1}{a})^\gamma \, (1-\epsilon) \; \; \textrm{if} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l\\ 
\frac{\epsilon}{|F|-1} \quad \quad \quad \textrm{if} \; t \; \textrm{is not mapped to} \; f \; \textrm{in} \; l
\end{array}
$$

where $f$ stands for form (in the sense of a complete signal, such as *aa*), $l$ stands for language, $t$ stands for topic, and $\epsilon$ stands for production error.

## How do we add *environmental* noise to this production model?


If we leave Kirby et al.'s production error aside for a moment, we could add environmental noise by allowing production to have two possible outcomes. Given a particular topic *t*, a speaker could produce either:
- form $f$ if $t$ is mapped to $f$ in $l$, with probability $1-n$
- form $f'$, where $f'$ is a 'noisy variant' of $f$ *and* $t$ is mapped to $f$ in $l$. This should happen with probability $\frac{1}{|F'|}$ (where $F'$ is the full set of possible noisy variants of the form $f$ that maps to $t$ in $l$

where $n$ stands for the probability of noise happening.

To give an example, using the example compositional grammar of Kirby et al. (2015):

S   --> A B   
A:0 --> a   
A:1 --> b   
B:2 --> a   
B:3 --> b   

Given this grammar, the only form that maps to topic 02 is *aa*.   
**Design decision:** If we allow for both 'partial' and 'full' noise,   
 this form has three possible noisy variants:
- a_
- _a
- \__

where _ stands for a noisy part of the signal that the listener could not perceive.

So if 02 is the speaker's intended topic (and if we exclude the possibility of the speaker making a production error for now), they will produce the following signals with the following probabilities:
- aa with probability $1-n$
- [a_, _a, \__ \] with probability $\frac{n}{3}$ (because there are 3 possible noisy variants


**Design decision:** should noisy variant \__ be less likely than other noisy variants, because it means that really *none* of the signal came through? Intuitively it would make sense if a_ was twice as likely to happen as \__, but that would make it a bit trickier to work out the probabilities for the different noisy variants. So to keep it simple I propose making all the noisy variants equally probable.




$$
P(f \mid l, t) \propto \Bigg\{
\begin{array}{ll}
(\frac{1}{a})^\gamma \, (1-\epsilon) \, (1-n) \; \; \textrm{if} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is intact} \\ 
(\frac{1}{a})^\gamma \, (1-\epsilon) \, n \quad \quad \; \; \textrm{if part of} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is not intact} \\ 
\frac{\epsilon}{|F|-1} \, (1-n) \quad \quad \quad \textrm{if} \; t \; \textrm{is not mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is intact}\\
\frac{\epsilon}{|F|-1} \, n \quad \quad \quad \quad \quad \textrm{if no part of} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is not intact}\\
\end{array}
$$

where *n* stands for the probability of noise.

### First, let's reproduce Kirby et al.'s (2015) model of production:

In order to do that we first need to set some general parameters:

In [22]:
meanings = ['02', '03', '12', '13'] # all possible meanings
forms = ['aa', 'ab', 'ba', 'bb'] # all possible forms
error = 0.05 # the probability of making a production error

Then we need some functions to generate all possible languages and classify languages according to the categories specified by Kirby et al. (2015:

In [27]:
import itertools

def create_all_possible_languages(meanings, forms):
    """Creates all possible languages

    :param meanings: list of strings corresponding to all possible meanings
    :type meanings: list
    :param forms: list of strings corresponding to all possible forms
    :type forms: list
    :returns: list of tuples which represent languages, where each tuple consists of forms and has length len(meanings)
    :rtype: list
    """
    all_possible_languages = list(itertools.combinations_with_replacement(forms, len(meanings)))
    return all_possible_languages


def classify_language(lang, forms, meanings):
    """
    Classify a given language as either 'compositional', 'holistic', 'degenerate' or 'other' (Kirby et al., 2015)

    :param lang: a language; represented as a tuple of forms, where each form index maps to same index in meanings
    :type lang: tuple
    :param forms: list of strings corresponding to all possible forms
    :type forms: list
    :returns: the category that the language falls into
    :rtype: str
    """
    #TODO: See if I can modify this function so that it can deal with any number of forms and meanings.
    # First check whether some conditions are met, bc this function hasn't been coded up in the most general way yet:
    if len(forms) != 4:
        raise ValueError("This function only works for a world in which there are 4 possible forms")
    if len(forms[0]) != 2:
        raise ValueError("This function only works when each form consists of 2 elements")
    if len(lang) != len(meanings):
        raise ValueError("Lang should have same length as meanings")
        
    # lang is degenerate if it uses the same form for every meaning:
    if lang[0] == lang[1] and lang[1] == lang[2] and lang[2] == lang[3]:
        return 'degenerate'
        
    # lang is compositional if it's *not* degenerate, and each form element maps to the same meaning element for each form:
    elif lang[0][0] == lang[1][0] and lang[2][0] == lang[3][0] and lang[0][1] == lang[2][1] and lang[1][1] == lang[3][1]:
        return 'compositional'
    
    # lang is holistic if it is *not* compositional, but *does* make us of all possible forms:
    elif forms[0] in lang and forms[1] in lang and forms[2] in lang and forms[3] in lang:
        return 'holistic'
    
    # In all other cases, a language belongs to the 'other' category:
    else:
        return 'other'
        
    
    
# Let's try out our create_all_possible_languages() function:
all_possible_languages = create_all_possible_languages(meanings, forms)
print("all_possible_languages are:")
print(all_possible_languages)
print("number of possible languages is:")
print(len(all_possible_languages))


# Let's test our classify_language() function using some example languages from the Kirby et al. (2015) paper:
degenerate_lang = ('aa', 'aa', 'aa', 'aa')
print('')
print("degenerate_lang is:")
print(degenerate_lang)
class_degenerate_lang = classify_language(degenerate_lang, forms, meanings)
print("class_degenerate_lang is:")
print(class_degenerate_lang)

compositional_lang = ('aa', 'ab', 'ba', 'bb')
print('')
print("compositional_lang is:")
print(compositional_lang)
class_compositional_lang = classify_language(compositional_lang, forms, meanings)
print("class_compositional_lang is:")
print(class_compositional_lang)

holistic_lang = ('aa', 'ab', 'bb', 'ba')
print('')
print("holistic_lang is:")
print(holistic_lang)
class_holistic_lang = classify_language(holistic_lang, forms, meanings)
print("class_holistic_lang is:")
print(class_holistic_lang)

other_lang = ('aa', 'aa', 'aa', 'ab')
print('')
print("other_lang is:")
print(other_lang)
class_other_lang = classify_language(other_lang, forms, meanings)
print("class_other_lang is:")
print(class_other_lang)

all_possible_languages are:
[('aa', 'aa', 'aa', 'aa'), ('aa', 'aa', 'aa', 'ab'), ('aa', 'aa', 'aa', 'ba'), ('aa', 'aa', 'aa', 'bb'), ('aa', 'aa', 'ab', 'ab'), ('aa', 'aa', 'ab', 'ba'), ('aa', 'aa', 'ab', 'bb'), ('aa', 'aa', 'ba', 'ba'), ('aa', 'aa', 'ba', 'bb'), ('aa', 'aa', 'bb', 'bb'), ('aa', 'ab', 'ab', 'ab'), ('aa', 'ab', 'ab', 'ba'), ('aa', 'ab', 'ab', 'bb'), ('aa', 'ab', 'ba', 'ba'), ('aa', 'ab', 'ba', 'bb'), ('aa', 'ab', 'bb', 'bb'), ('aa', 'ba', 'ba', 'ba'), ('aa', 'ba', 'ba', 'bb'), ('aa', 'ba', 'bb', 'bb'), ('aa', 'bb', 'bb', 'bb'), ('ab', 'ab', 'ab', 'ab'), ('ab', 'ab', 'ab', 'ba'), ('ab', 'ab', 'ab', 'bb'), ('ab', 'ab', 'ba', 'ba'), ('ab', 'ab', 'ba', 'bb'), ('ab', 'ab', 'bb', 'bb'), ('ab', 'ba', 'ba', 'ba'), ('ab', 'ba', 'ba', 'bb'), ('ab', 'ba', 'bb', 'bb'), ('ab', 'bb', 'bb', 'bb'), ('ba', 'ba', 'ba', 'ba'), ('ba', 'ba', 'ba', 'bb'), ('ba', 'ba', 'bb', 'bb'), ('ba', 'bb', 'bb', 'bb'), ('bb', 'bb', 'bb', 'bb')]
number of possible languages is:
35

degenerate_lang is:
('aa

Now let's implement the actual production function. Here is the equation from Kirby et al. (2015) again:

$$
P(f \mid l, t) \propto \Bigg\{
\begin{array}{ll}
(\frac{1}{a})^\gamma \, (1-\epsilon) \; \; \textrm{if} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l\\ 
\frac{\epsilon}{|F|-1} \quad \quad \quad \textrm{if} \; t \; \textrm{is not mapped to} \; f \; \textrm{in} \; l
\end{array}
$$

where $f$ stands for form (in the sense of a complete signal, such as *aa*), $l$ stands for language, $t$ stands for topic, and $\epsilon$ stands for production error.

In [29]:
def production(language, topic):
    print('')
    print('This is the production() function at work:')
    print('')
    print("language is:")
    print(language)
    print('')
    print("meanings are:")
    print(meanings)
    for m in range(len(meanings)):
        if meanings[m] == topic:
            topic_index = m
    print('')
    print("topic_index is:")
    print(topic_index)
    correct_form = language[topic_index]
    print("correct_form is:")
    print(correct_form)
    error_forms = list(language)
    error_forms.remove(correct_form)
    print("error_forms is:")
    print(error_forms)
    print("Check that removing the correct form didn't mess up the language:")
    print("language is:")
    print(language)
    


production(compositional_lang, "02")


This is the production() function at work:

language is:
('aa', 'ab', 'ba', 'bb')

meanings are:
['02', '03', '12', '13']

topic_index is:
0
correct_form is:
aa
error_forms is:
['ab', 'ba', 'bb']
Check that removing the correct form didn't mess up the language:
language is:
('aa', 'ab', 'ba', 'bb')
